In [ ]:
!pip install findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 57.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2d9b3b9b726ea66b412258a8c5dd28b05ebd87c6691429ff20c7255e155ba6a6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import findspark
import pandas as pd
findspark.init()
from pyspark.sql import SparkSession

def init_spark(app_name: str):
  spark = SparkSession.builder.appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

In [ ]:
spark, sc = init_spark('demo')
sc

In [ ]:
from pyspark.sql.types import *
import re
from pyspark.sql import functions as f

In [ ]:
spark1 = SparkSession.builder.appName('Users_DF').getOrCreate()


user_schema = StructType([
                    StructField("user_id", IntegerType()),
                    StructField('user_location', StringType())])
                     
dataPath = "users.csv"
users_df = spark1.read.format("csv")\
  .option("header","true")\
  .schema(user_schema)\
  .load(dataPath)
users_df = users_df.withColumn('user_id', f.col('user_id').cast('string'))

In [ ]:
dataPath2 = "movies.csv"
movies_df = spark1.read.format("csv")\
  .option("header","true")\
  .option("escape", "\"")\
  .option('inferSchema', "true")\
  .load(dataPath2)


prog = re.compile("(name': ')((((\w+|&|\/) |((\w+|&|\/)([^(\'\")](\, )*)( )*|\((.*?)\)( )*|\"(.*?)\"|\d+)+)+))(')")
second_match = f.udf(lambda x: [f"{i[1]}" for i in prog.findall(x)], ArrayType(StringType()))

prog2 = re.compile("(')(\w+)(')")
cities_udf = f.udf(lambda x: [i[1] for i in prog2.findall(x)], ArrayType(StringType()))

prog3 = re.compile("(name': ')(.*?)('})")
lang_udf = f.udf(lambda x: [i[1] for i in prog3.findall(x)], ArrayType(StringType()))

prog4 = re.compile("\d+")
id_udf = f.udf(lambda x: prog4.findall(x), StringType())

for col_name in ['genres','production_companies','production_countries']:
    movies_df = movies_df.withColumn(col_name, second_match(col_name))

movies_df = movies_df.withColumn("release_date",f.split(f.col("release_date"), "/").getItem(2))\
  .withColumn('cities', cities_udf('cities'))\
  .withColumn('movie_id', id_udf('movie_id'))\
  .withColumn("movie_id", f.expr("substring(movie_id, 2, length(movie_id)-2)"))\
  .withColumn('lang', lang_udf('spoken_languages'))\
  .drop('revenue', 'tagline', 'spoken_languages', 'overview')

In [ ]:
dataPath3 = "tickets.csv"
tickets_df = spark1.read.format("csv")\
  .option("header","true")\
  .option("encoding","UTF-8")\
  .option("inferSchema", "true")\
  .load(dataPath3)

tickets_df = tickets_df.withColumn('user_id', f.col('user_id').cast('string'))\
            .withColumn('movie_id', f.col('movie_id').cast('string'))\
            .withColumn('cinema_id', f.col('cinema_id').cast('string'))\

In [ ]:
dataPath5 = "queries.csv"
queries_df = spark1.read.format("csv")\
  .option("header","true")\
  .option("escape", "\"")\
  .option("encoding","UTF-8")\
  .option("inferSchema", "true")\
  .load(dataPath5)


list_prog = re.compile(("(((\w+|&|\/) |((\w+|&|\/)([^(\'\")](\, )*)( )*|\((.*?)\)( )*|\"(.*?)\"|\d+)+)+)"))
list_udf = f.udf(lambda x: [i[0] for i in list_prog.findall(x)], ArrayType(StringType()))

q_lang_prog = re.compile("(')(.*?)(')")
q_lang_udf = f.udf(lambda x: [i[1] for i in q_lang_prog.findall(x)], ArrayType(StringType()))

for col_name in ['actors','director','cities', 'country','production_company','genres']:
    queries_df = queries_df.withColumn(col_name, list_udf(col_name))
queries_df = queries_df.withColumn("realease_date", f.expr("substring(from_realese_date, 3, length(from_realese_date)-4)"))\
    .withColumn('lang', q_lang_udf('lang'))\
    .withColumn('user_id', f.col('user_id').cast('string'))\
    .drop('from_realese_date')

In [ ]:
dataPath6 = "credits.csv"
credits_df = spark1.read.format("csv")\
  .option("header","true")\
  .option("escape", "\"")\
  .option("encoding","UTF-8")\
  .option("inferSchema", "true")\
  .load(dataPath6)

cast_prog = re.compile("(name)(': ')((((\w+|&|\/) |((\w+|&|\/)([^(\'\")](\, )*)( )*|\((.*?)\)( )*|\"(.*?)\"|\d+)+)+))")
cast_udf = f.udf(lambda x: [f"{i[2]}" for i in cast_prog.findall(x)], ArrayType(StringType()))

crew_prog = re.compile("('Director\\', \\'name\\': \\')((((\w+|&|\/) |((\w+|&|\/)([^(\'\")](\, )*)( )*|\((.*?)\)( )*|\"(.*?)\"|\d+)+)+))(\\',)")
crew_udf = f.udf(lambda x: [f"{i[1]}" for i in crew_prog.findall(x)], ArrayType(StringType()))

credits_df = credits_df.withColumn('actors', cast_udf('cast')).withColumn('director', crew_udf('crew')).drop('crew', 'cast')

ANALIZE:

In [ ]:
def check_null(df):
  df.select([f.count(f.when(f.col(c) == '[]', c)).alias(c) for c in df.columns]).show()

In [ ]:
temp_path = "queries.csv"
temp_df = spark1.read.format("csv")\
  .option("header","true")\
  .option("escape", "\"")\
  .option("encoding","UTF-8")\
  .option("inferSchema", "true")\
  .load(temp_path)
check_null(temp_df)

+-------+------+----+------+--------+------+-------+-----------------+------------------+
|user_id|genres|lang|actors|director|cities|country|from_realese_date|production_company|
+-------+------+----+------+--------+------+-------+-----------------+------------------+
|      0|   141|   0| 44701|   77688|     0|  45071|                0|                 0|
+-------+------+----+------+--------+------+-------+-----------------+------------------+



We notice that out of the possible 100,000 tuples, about half don't care about specific actors or production country.
This is especially the case with directors - nore then 75% of queries don't consider the director of the movies.

In [ ]:
queries_df.withColumn('lang', f.explode('lang'))\
    .withColumn('cities', f.explode('cities'))\
    .select('*').where(f.col('lang') == 'עִבְרִית').groupBy('lang','cities').count().orderBy('count', ascending=False).show()

+--------+---------+-----+
|    lang|   cities|count|
+--------+---------+-----+
|עִבְרִית|Jerusalem|26274|
|עִבְרִית| Tel Aviv|15958|
|עִבְרִית|    Haifa|  104|
|עִבְרִית|    Eilat|   67|
|עִבְרִית| Tiberias|   56|
+--------+---------+-----+



In [ ]:
queries_df.select('*').groupBy('cities').count().orderBy('count', ascending=False).take(10)

[Row(cities=['Tel Aviv', 'Jerusalem'], count=22276),
 Row(cities=['Tel Aviv'], count=21151),
 Row(cities=['Eilat'], count=17348),
 Row(cities=['Haifa'], count=17346),
 Row(cities=['Jerusalem'], count=12285),
 Row(cities=['Tiberias'], count=4228),
 Row(cities=['Haifa', 'Tiberias'], count=4072),
 Row(cities=['Haifa', 'Tiberias', 'Tel Aviv'], count=1294)]

In [ ]:
user_q_df.select('*').groupBy('user_location').count().orderBy('count', ascending=False).take(10)

[Row(user_location='Jerusalem', count=30873),
 Row(user_location='Tel Aviv', count=24839),
 Row(user_location='Haifa', count=21652),
 Row(user_location='Eilat', count=17348),
 Row(user_location='Kibuts Gesher', count=5288)]

In [ ]:
users_df.select('*').groupBy('user_location').count().take(5)

[Row(user_location='Kibuts Gesher', count=576),
 Row(user_location='Haifa', count=2324),
 Row(user_location='Jerusalem', count=3263),
 Row(user_location='Tel Aviv', count=2643),
 Row(user_location='Eilat', count=1850)]

In [ ]:
tickets_u_df = tickets_df.join(users_df, on='user_id')

In [ ]:
tickets_u_df.show()

In [ ]:
users_q_df = queries_df.join(users_df, on='user_id')

In [ ]:
tickets_u_df.select('*').groupBy('user_location').sum('number_of_tickets').orderBy('sum(number_of_tickets)', ascending=False).take(25)

[Row(user_location='Jerusalem', sum(number_of_tickets)=315926),
 Row(user_location='Haifa', sum(number_of_tickets)=228375),
 Row(user_location='Tel Aviv', sum(number_of_tickets)=140203),
 Row(user_location='Eilat', sum(number_of_tickets)=138043),
 Row(user_location='Kibuts Gesher', sum(number_of_tickets)=27915)]

In [ ]:
users_q_df.select('*').where(f.col('director')[0].isNotNull()).groupBy('user_location').count().orderBy('count', ascending=False).take(25)

[Row(user_location='Haifa', count=19848),
 Row(user_location='Jerusalem', count=963),
 Row(user_location='Tel Aviv', count=763),
 Row(user_location='Eilat', count=530),
 Row(user_location='Kibuts Gesher', count=151)]

In [ ]:
users_q_df.select('*').where(f.col('actors')[0].isNotNull()).groupBy('user_location').count().orderBy('count', ascending=False).take(25)

[Row(user_location='Jerusalem', count=29355),
 Row(user_location='Tel Aviv', count=12509),
 Row(user_location='Haifa', count=10756),
 Row(user_location='Kibuts Gesher', count=2625)]

In [ ]:
users_q_df.select('*').where(f.col('country')[0].isNotNull()).groupBy('user_location').count().orderBy('count', ascending=False).take(25)

[Row(user_location='Jerusalem', count=30873),
 Row(user_location='Eilat', count=17348),
 Row(user_location='Kibuts Gesher', count=5288),
 Row(user_location='Tel Aviv', count=760),
 Row(user_location='Haifa', count=660)]

In [ ]:
movies_df.groupBy('release_date').count().orderBy('count', ascending=False).take(25)

Algorithm for VF deciding which attribute to seperate - between genres, director, actirs and country.

Step 1 : creating queries frequencies matrix

In [ ]:
func_df = queries_df.join(users_df, on='user_id').select('country', 'actors', 'director','genres', 'user_location')
not_c = f.col('country')[0].isNull()
c = f.col('country')[0].isNotNull()
not_a = f.col('actors')[0].isNull()
a = f.col('actors')[0].isNotNull()
not_d = f.col('director')[0].isNull()
d = f.col('director')[0].isNotNull()
not_g = f.col('genres')[0].isNull()
g = f.col('genres')[0].isNotNull()
dict_q = {}
q1 =  func_df.select('*').where(not_a & not_c & not_d & not_g).groupBy('user_location').count().collect()
q2 = func_df.select('*').where(a & not_c & not_d & not_g).groupBy('user_location').count().collect()
q3 = func_df.select('*').where(not_a & c & not_d & not_g).groupBy('user_location').count().collect()
q4 = func_df.select('*').where(not_a & not_c & d & not_g).groupBy('user_location').count().collect()
q5 = func_df.select('*').where(a & c & not_d & not_g).groupBy('user_location').count().collect()
q6 = func_df.select('*').where(a & not_c & d & not_g).groupBy('user_location').count().collect()
q7 = func_df.select('*').where(not_a & c & d & not_g).groupBy('user_location').count().collect()
q8 = func_df.select('*').where(a & c & d & not_g).groupBy('user_location').count().collect()
q9 =  func_df.select('*').where(not_a & not_c & not_d & g).groupBy('user_location').count().collect()
q10 = func_df.select('*').where(a & not_c & not_d & g).groupBy('user_location').count().collect()
q11 = func_df.select('*').where(not_a & c & not_d & g).groupBy('user_location').count().collect()
q12 = func_df.select('*').where(not_a & not_c & d & g).groupBy('user_location').count().collect()
q13 = func_df.select('*').where(a & c & not_d & g).groupBy('user_location').count().collect()
q14 = func_df.select('*').where(a & not_c & d & g).groupBy('user_location').count().collect()
q15 = func_df.select('*').where(not_a & c & d & g).groupBy('user_location').count().collect()
q16 = func_df.select('*').where(a & c & d & g).groupBy('user_location').count().collect()




Creating the frequencies matrix, and matrix q_per_att describes which attributes every query access.

In [ ]:
sites = ['Jerusalem', 'Haifa', 'Tel Aviv','Eilat','Kibuts Gesher']
att = ['actors','country', 'director', 'genres']
q_per_site = pd.DataFrame(columns = sites)
q_per_att = pd.DataFrame(columns = att, index=range(1,17))
index = 1
for q in [q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16]:
  temp_dict = {}
  for site in sites:
      temp_dict[site] = 0
  for i in q:
    temp_dict[i['user_location']] = i['count']
  temp_df = pd.DataFrame(temp_dict, index=[index])
  index += 1
  q_per_site = pd.concat([q_per_site, temp_df])

q_per_att.iloc[0] = [0,0,0,0]
q_per_att.iloc[1] = [1,0,0,0]
q_per_att.iloc[2] = [0,1,0,0]
q_per_att.iloc[3] = [0,0,1,0]
q_per_att.iloc[4] = [1,1,0,0]
q_per_att.iloc[5] = [1,0,1,0]
q_per_att.iloc[6] = [0,1,1,0]
q_per_att.iloc[7] = [1,1,1,0]
q_per_att.iloc[8] = [0,0,0,1]
q_per_att.iloc[9] = [1,0,0,1]
q_per_att.iloc[10] = [0,1,0,1]
q_per_att.iloc[11] = [0,0,1,1]
q_per_att.iloc[12] = [1,1,0,1]
q_per_att.iloc[13] = [1,0,1,1]
q_per_att.iloc[14] = [0,1,1,1]
q_per_att.iloc[15] = [1,1,1,1]

Frequencies Matrix:

In [ ]:
q_per_site

,Jerusalem,Haifa,Tel Aviv,Eilat,Kibuts Gesher
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,64
4,0,0,0,0,0
5,0,0,0,0,70
6,0,0,0,0,0
7,0,0,0,0,3
8,0,0,0,0,4
9,0,882,11584,0,0
10,0,867,11757,0,0


Attributes accessed Matrix:

** Note we assume accessibility of every query to every attribute is 1.

In [ ]:
q_per_att

,actors,country,director,genres
1,0,0,0,0
2,1,0,0,0
3,0,1,0,0
4,0,0,1,0
5,1,1,0,0
6,1,0,1,0
7,0,1,1,0
8,1,1,1,0
9,0,0,0,1
10,1,0,0,1


Step 2: Compute AAM matrix.

Defining local function for calculating affinity.

In [ ]:
def affinity(i,j):
  sum1 = 0
  for k in range(16):
    if q_per_att.iloc[k][i] == 1 and q_per_att.iloc[k][j] == 1:
      sum1 += sum(q_per_site.iloc[k])
  return sum1

Compute AAM Matrix:

In [ ]:
import numpy as np
AAM = np.zeros((4,4))
for i, att in enumerate(['actors','country', 'director', 'genres']):
  for j, att2 in enumerate(['actors','country', 'director', 'genres']):
      AAM[i][j] = affinity(i,j)


The AAM matrix:

In [ ]:
AAM

array([[55245., 32685., 11243., 55171.],
       [32685., 54929.,  2274., 54788.],
       [11243.,  2274., 22255., 22248.],
       [55171., 54788., 22248., 99859.]])

Compute and change order of AAM using Bond Energy Algorithm.

In [ ]:
import numpy as np
query_attr1 = q_per_att.to_numpy()
query_access1 = q_per_site.to_numpy()
query_attr = q_per_att.to_numpy()
query_access = q_per_site.to_numpy()
aa_matrix = np.zeros((4,4))

def position_checker(positions):
  aa_elem = []
  for j in positions[0]:
    aa_elem.append(np.sum(query_access[j:j+1, :]))
  aa_matrix[i,k] = sum(aa_elem)

def dot_prod(vec_a, vec_b):
  return np.dot(vec_a.flatten(), vec_b.flatten())

## AA_MARIX GENERATION
for i in range(4):
  for k in range(4):
    if i == k:
      attr = query_attr[:, k:k+1].flatten()
      positions = np.where(attr == 1)
      position_checker(positions)
    else:
      attr_i = query_attr[:, i:i+1].flatten()
      attr_k = query_attr[:, k:k+1].flatten()
      positions = np.where((attr_i == 1) & (attr_k == 1))
      position_checker(positions)

# BOND ENERGY ALGORITHM - GET INITIAL CC_MATRIX
cc_matrix = aa_matrix[:, :2]
cc_matrix = np.append(np.zeros((4,1)), cc_matrix, axis=1)
cc_matrix = np.append(cc_matrix,np.zeros((4,1)), axis=1)

# BOND ENERGY ALGORITHM - CC_MATRIX GENERATION
for k in range(2, len(aa_matrix)):
  i, j = 0, 1
  get_values = []
  while j < cc_matrix.shape[1]:
    print(i,k,j, len(cc_matrix))
    get_values.append(2 * dot_prod(cc_matrix[:, i:i+1], aa_matrix[:, k:k+1]) + 
                      2 * dot_prod(aa_matrix[:, k:k+1], cc_matrix[:, j:j+1]) - 
                      2 * dot_prod(cc_matrix[:, i:i+1], cc_matrix[:, j:j+1]))  
    i = j
    j+=1
  pos = np.argmax(get_values) + 1
  cc_matrix = np.insert(cc_matrix, pos, aa_matrix[k:k+1], axis=1)

# BOND ENERGY ALGORITHM - REMOVES ZEROS COLUMNS
cc_matrix = np.delete(cc_matrix, 0, axis=1)
cc_matrix = np.delete(cc_matrix, cc_matrix.shape[1]-1, axis=1)

print(cc_matrix)

0 2 1 4
1 2 2 4
2 2 3 4
0 3 1 4
1 3 2 4
2 3 3 4
3 3 4 4
[[11243. 55245. 55171. 32685.]
 [ 2274. 32685. 54788. 54929.]
 [22255. 11243. 22248.  2274.]
 [22248. 55171. 99859. 54788.]]


Transform CC matrix to being symmetrical

In [ ]:
for i in range(cc_matrix.shape[1]):
  for j in range(i):
    cc_matrix[j,i] = cc_matrix[i,j] 

The CC Matrix:

In [ ]:
cc_matrix

array([[11243.,  2274., 22255., 22248.],
       [ 2274., 32685., 11243., 55171.],
       [22255., 11243., 22248., 99859.],
       [22248., 55171., 99859., 54788.]])

So The order by attributes in both matrixes are:

AAM : ['actors','country', 'director', 'genres']

CC_MATRIX = ['director', 'actors', 'genres','country']

- Configuring indexes of queries. for example: q_a are all queries accessing actors.

- We can now compute the groups TA,BA,QA (we changed the names instead of TQ,BQ,OQ).

- sum_of_q = sum of every query asked. we have the indexes in the groups TA,BA,QA, and then we can compute CTA,CBA,CQA.

- lastly we print the value  CTA*CBA - CQA^2 and choose the maximal.

- We then change the order of columns until we find the best split.

Order: Director, Actors, Genres, Country

In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_d).difference(set(q_g).union(set(q_c).union(set(q_a))))
BA = set(q_g).union(set(q_c)).union(set(q_a)).difference(set(q_d))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))


-495285025.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_d).union(set(q_a)).difference(set(q_g).union(set(q_c)))
BA = set(q_c).union(set(q_g)).difference(set(q_a).union(set(q_d)))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))




-4389990049.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_d).union(set(q_a).union(set(q_g))).difference(set(q_c))
BA = set(q_c).difference(set(q_d).union(set(q_a).union(set(q_g))))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))


-3007283681.0


Order: Actors, Genres, Country, Director



In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_a).difference(set(q_g).union(set(q_c).union(set(q_d))))
BA = set(q_g).union(set(q_c)).union(set(q_d)).difference(set(q_a))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))


-3052010025.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_g).union(set(q_a)).difference(set(q_d).union(set(q_c)))
BA = set(q_c).union(set(q_d)).difference(set(q_a).union(set(q_g)))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))


-5599793619.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_c).union(set(q_a).union(set(q_g))).difference(set(q_d))
BA = set(q_d).difference(set(q_c).union(set(q_a).union(set(q_g))))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-495285025.0


Order: Genres, Country, Director, Actors



In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_g).difference(set(q_a).union(set(q_c).union(set(q_d))))
BA = set(q_a).union(set(q_c)).union(set(q_d)).difference(set(q_g))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-7635778743.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_g).union(set(q_c)).difference(set(q_d).union(set(q_a)))
BA = set(q_a).union(set(q_d)).difference(set(q_c).union(set(q_g)))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-4389990049.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_c).union(set(q_d).union(set(q_g))).difference(set(q_a))
BA = set(q_a).difference(set(q_c).union(set(q_d).union(set(q_g))))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-3052010025.0


Order: Country, Director, Actors, Genres



In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_c).difference(set(q_a).union(set(q_g).union(set(q_d))))
BA = set(q_a).union(set(q_g)).union(set(q_d)).difference(set(q_c))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-3007283681.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_d).union(set(q_c)).difference(set(q_g).union(set(q_a)))
BA = set(q_a).union(set(q_g)).difference(set(q_c).union(set(q_d)))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-5599793619.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8,10,13,14,16]
q_d = [4,6,7,8,12,14,15,16]
q_c = [3,5,7,8,11,13,15,16]
q_g =  [9,10,11,12,13,14,15,16]
TA = set(q_c).union(set(q_d).union(set(q_a))).difference(set(q_g))
BA = set(q_g).difference(set(q_c).union(set(q_d).union(set(q_a))))
QA = set([i for i in range(1,17)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-7635778743.0


In [ ]:
min(queries_df.select('realease_date').distinct().collect())

In [ ]:
year_count = []
for i in range(1990,2019):
  rd = str(i)
  temp_count =  users_q_df.select('*').where((f.col('realease_date') < rd) & (f.col('user_location') == 'Tel Aviv')).count()
  year_count.append((rd,temp_count))
print(year_count)

[('1990', 0), ('1991', 0), ('1992', 0), ('1993', 0), ('1994', 0), ('1995', 0), ('1996', 0), ('1997', 0), ('1998', 0), ('1999', 0), ('2000', 0), ('2001', 0), ('2002', 0), ('2003', 0), ('2004', 0), ('2005', 0), ('2006', 0), ('2007', 0), ('2008', 0), ('2009', 0), ('2010', 0), ('2011', 199), ('2012', 410), ('2013', 3960), ('2014', 7490), ('2015', 11132), ('2016', 14771), ('2017', 18024), ('2018', 21377)]


In [ ]:
year_count1 = []
for i in range(1990,2019):
  rd = str(i)
  temp_count =  users_q_df.select('*').where((f.col('realease_date') < rd) & (f.col('user_location') == 'Haifa')).count()
  year_count1.append((rd,temp_count))
print(year_count1)

[('1990', 0), ('1991', 0), ('1992', 0), ('1993', 0), ('1994', 0), ('1995', 0), ('1996', 0), ('1997', 0), ('1998', 0), ('1999', 0), ('2000', 0), ('2001', 0), ('2002', 0), ('2003', 0), ('2004', 0), ('2005', 0), ('2006', 0), ('2007', 0), ('2008', 0), ('2009', 0), ('2010', 0), ('2011', 166), ('2012', 345), ('2013', 3432), ('2014', 6523), ('2015', 9721), ('2016', 12816), ('2017', 15801), ('2018', 18709)]


In [ ]:
year_count2 = []
for i in range(1990,2019):
  rd = str(i)
  temp_count =  users_q_df.select('*').where((f.col('realease_date') < rd) & (f.col('user_location') == 'Jerusalem')).count()
  year_count2.append((rd,temp_count))
print(year_count2)

[('1990', 0), ('1991', 1097), ('1992', 2160), ('1993', 3245), ('1994', 4317), ('1995', 5383), ('1996', 6481), ('1997', 7500), ('1998', 8613), ('1999', 9664), ('2000', 10713), ('2001', 11734), ('2002', 12853), ('2003', 13949), ('2004', 14976), ('2005', 16040), ('2006', 17097), ('2007', 18181), ('2008', 19248), ('2009', 20305), ('2010', 21369), ('2011', 22395), ('2012', 23472), ('2013', 24502), ('2014', 25604), ('2015', 26649), ('2016', 27710), ('2017', 28775), ('2018', 29821)]


In [ ]:
year_count3 = []
for i in range(1990,2019):
  rd = str(i)
  temp_count =  users_q_df.select('*').where((f.col('realease_date') < rd) & (f.col('user_location') == 'Eilat')).count()
  year_count3.append((rd,temp_count))
print(year_count3)

[('1990', 0), ('1991', 606), ('1992', 1140), ('1993', 1700), ('1994', 2322), ('1995', 2960), ('1996', 3522), ('1997', 4121), ('1998', 4725), ('1999', 5307), ('2000', 5885), ('2001', 6497), ('2002', 7108), ('2003', 7712), ('2004', 8349), ('2005', 8922), ('2006', 9542), ('2007', 10139), ('2008', 10728), ('2009', 11307), ('2010', 11946), ('2011', 12542), ('2012', 13144), ('2013', 13748), ('2014', 14345), ('2015', 14958), ('2016', 15531), ('2017', 16097), ('2018', 16743)]


In [ ]:
year_count4 = []
for i in range(1990,2019):
  rd = str(i)
  temp_count =  users_q_df.select('*').where((f.col('realease_date') < rd) & (f.col('user_location') == 'Kibuts Gesher')).count()
  year_count4.append((rd,temp_count))
print(year_count4)

[('1990', 0), ('1991', 201), ('1992', 430), ('1993', 700), ('1994', 941), ('1995', 1161), ('1996', 1373), ('1997', 1618), ('1998', 1847), ('1999', 2088), ('2000', 2315), ('2001', 2567), ('2002', 2774), ('2003', 2990), ('2004', 3233), ('2005', 3447), ('2006', 3673), ('2007', 3904), ('2008', 4141), ('2009', 4380), ('2010', 4605), ('2011', 4824), ('2012', 5056), ('2013', 5288), ('2014', 5288), ('2015', 5288), ('2016', 5288), ('2017', 5288), ('2018', 5288)]


Algorithm for VF deciding which attribute to seperate - between genres, actors and country.

Step 1 : creating queries frequencies matrix

In [ ]:
func_df = queries_df.join(users_df, on='user_id').select('country', 'actors', 'genres', 'user_location')
not_c = f.col('country')[0].isNull()
c = f.col('country')[0].isNotNull()
not_a = f.col('actors')[0].isNull()
a = f.col('actors')[0].isNotNull()
not_g = f.col('genres')[0].isNull()
g = f.col('genres')[0].isNotNull()
dict_q = {}
q1 =  func_df.select('*').where(not_a & not_c & not_g).groupBy('user_location').count().collect()
q2 = func_df.select('*').where(a & not_c & not_g).groupBy('user_location').count().collect()
q3 =  func_df.select('*').where(not_a & c & not_g).groupBy('user_location').count().collect()
q4 = func_df.select('*').where(not_a & not_c & g).groupBy('user_location').count().collect()
q5 =  func_df.select('*').where(a & c & not_g).groupBy('user_location').count().collect()
q6 = func_df.select('*').where(a & not_c & g).groupBy('user_location').count().collect()
q7 =  func_df.select('*').where(not_a & c & g).groupBy('user_location').count().collect()
q8 = func_df.select('*').where(a & c & g).groupBy('user_location').count().collect()

Creating the frequencies matrix, and matrix q_per_att describes which attributes every query access.

In [ ]:
sites = ['Jerusalem', 'Haifa', 'Tel Aviv','Eilat','Kibuts Gesher']
att = ['actors','country', 'genres']
q_per_site = pd.DataFrame(columns = sites)
q_per_att = pd.DataFrame(columns = att, index=range(1,9))
index = 1
for q in [q1,q2,q3,q4,q5,q6,q7,q8]:
  temp_dict = {}
  for site in sites:
      temp_dict[site] = 0
  for i in q:
    temp_dict[i['user_location']] = i['count']
  temp_df = pd.DataFrame(temp_dict, index=[index])
  index += 1
  q_per_site = pd.concat([q_per_site, temp_df])

q_per_att.iloc[0] = [0,0,0]
q_per_att.iloc[1] = [1,0,0]
q_per_att.iloc[2] = [0,1,0]
q_per_att.iloc[3] = [0,0,1]
q_per_att.iloc[4] = [1,1,0]
q_per_att.iloc[5] = [1,0,1]
q_per_att.iloc[6] = [0,1,1]
q_per_att.iloc[7] = [1,1,1]

Frequencies Matrix:

In [ ]:
q_per_site

,Jerusalem,Haifa,Tel Aviv,Eilat,Kibuts Gesher
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,67
4,0,10536,11928,0,0
5,0,0,0,0,74
6,0,10456,12151,0,0
7,1518,336,378,17348,2590
8,29355,324,382,0,2557


Attributes accessed Matrix:

** Note we assume accessibility of every query to every attribute is 1.

In [ ]:
q_per_att

,actors,country,genres
1,0,0,0
2,1,0,0
3,0,1,0
4,0,0,1
5,1,1,0
6,1,0,1
7,0,1,1
8,1,1,1


Step 2: Compute AAM matrix.

Defining local function for calculating affinity.

In [ ]:
def affinity(i,j):
  sum1 = 0
  for k in range(8):
    if q_per_att.iloc[k][i] == 1 and q_per_att.iloc[k][j] == 1:
      sum1 += sum(q_per_site.iloc[k])
  return sum1

Compute AAM Matrix:

In [ ]:
import numpy as np
AAM = np.zeros((3,3))
for i, att in enumerate(['actors','country', 'genres']):
  for j, att2 in enumerate(['actors','country', 'genres']):
      AAM[i][j] = affinity(i,j)


The AAM matrix:

In [ ]:
AAM

array([[55299., 32692., 55225.],
       [32692., 54929., 54788.],
       [55225., 54788., 99859.]])

Compute and change order of AAM using Bond Energy Algorithm.

In [ ]:
import numpy as np
query_attr1 = q_per_att.to_numpy()
query_access1 = q_per_site.to_numpy()
query_attr = q_per_att.to_numpy()
query_access = q_per_site.to_numpy()
aa_matrix = np.zeros((3,3))

def position_checker(positions):
  aa_elem = []
  for j in positions[0]:
    aa_elem.append(np.sum(query_access[j:j+1, :]))
  aa_matrix[i,k] = sum(aa_elem)

def dot_prod(vec_a, vec_b):
  return np.dot(vec_a.flatten(), vec_b.flatten())

## AA_MARIX GENERATION
for i in range(3):
  for k in range(3):
    if i == k:
      attr = query_attr[:, k:k+1].flatten()
      positions = np.where(attr == 1)
      position_checker(positions)
    else:
      attr_i = query_attr[:, i:i+1].flatten()
      attr_k = query_attr[:, k:k+1].flatten()
      positions = np.where((attr_i == 1) & (attr_k == 1))
      position_checker(positions)

# BOND ENERGY ALGORITHM - GET INITIAL CC_MATRIX
cc_matrix = aa_matrix[:, :2]
cc_matrix = np.append(np.zeros((3,1)), cc_matrix, axis=1)
cc_matrix = np.append(cc_matrix,np.zeros((3,1)), axis=1)

# BOND ENERGY ALGORITHM - CC_MATRIX GENERATION
for k in range(2, len(aa_matrix)):
  i, j = 0, 1
  get_values = []
  while j < cc_matrix.shape[1]:
    print(i,k,j, len(cc_matrix))
    get_values.append(2 * dot_prod(cc_matrix[:, i:i+1], aa_matrix[:, k:k+1]) + 
                      2 * dot_prod(aa_matrix[:, k:k+1], cc_matrix[:, j:j+1]) - 
                      2 * dot_prod(cc_matrix[:, i:i+1], cc_matrix[:, j:j+1]))  
    i = j
    j+=1
  pos = np.argmax(get_values) + 1
  cc_matrix = np.insert(cc_matrix, pos, aa_matrix[k:k+1], axis=1)

# BOND ENERGY ALGORITHM - REMOVES ZEROS COLUMNS
cc_matrix = np.delete(cc_matrix, 0, axis=1)
cc_matrix = np.delete(cc_matrix, cc_matrix.shape[1]-1, axis=1)

print(cc_matrix)

0 2 1 3
1 2 2 3
2 2 3 3
[[55299. 55225. 32692.]
 [32692. 54788. 54929.]
 [55225. 99859. 54788.]]


Transform CC matrix to being symmetrical

In [ ]:
for i in range(cc_matrix.shape[1]):
  for j in range(i):
    cc_matrix[j,i] = cc_matrix[i,j] 

The CC Matrix:

In [ ]:
cc_matrix

array([[55299., 55225., 32692.],
       [32692., 54788., 54929.],
       [55225., 99859., 54788.]])

So The order by attributes in both matrixes are:

AAM : ['actors','country', 'genres']

CC_MATRIX = ['actors', 'genres','country']

- Configuring indexes of queries. for example: q_a are all queries accessing actors.

- We can now compute the groups TA,BA,QA (we changed the names instead of TQ,BQ,OQ).

- sum_of_q = sum of every query asked. we have the indexes in the groups TA,BA,QA, and then we can compute CTA,CBA,CQA.

- lastly we print the value  CTA*CBA - CQA^2 and choose the maximal.

- We then change the order of columns until we find the best split.

Order = Actors, Genres, Country

In [ ]:
q_per_site.sum(axis=1)

1        0.0
2        0.0
3       67.0
4    22464.0
5       74.0
6    22607.0
7    22170.0
8    32618.0
dtype: float64

In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_a).difference(set(q_c).union(set(q_g)))
BA = set(q_c).union(set(q_g)).difference(set(q_a))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-3057979401.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_a).union(set(q_c)).difference(set(q_g))
BA = set(q_g).difference(set(q_a).union(set(q_c)))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-5986818601.0


Order = Country, Actors, Genres

In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_c).difference(set(q_a).union(set(q_g)))
BA = set(q_g).union(set(q_a)).difference(set(q_c))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-3006819287.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_c).union(set(q_a)).difference(set(q_g))
BA = set(q_g).difference(set(q_c).union(set(q_a)))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-5986818601.0


Order = Genres,Country, Actors

In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_g).difference(set(q_c).union(set(q_a)))
BA = set(q_c).union(set(q_a)).difference(set(q_g))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-5986818601.0


In [ ]:
sum_of_q = q_per_site.sum(axis=1)
q_a = [2,5,6,8]
q_c = [3,5,7,8]
q_g =  [4,6,7,8]
TA = set(q_g).union(set(q_c)).difference(set(q_a))
BA = set(q_a).difference(set(q_c).union(set(q_g)))
QA = set([i for i in range(1,9)]).difference(TA.union(BA))
CTA = sum([sum_of_q.iloc[i-1] for i in TA])
CBA = sum([sum_of_q.iloc[i-1] for i in BA])
CQA = sum([sum_of_q.iloc[i-1] for i in QA])
print(CTA*CBA-(CQA**2))

-3057979401.0
